In [98]:
import os
import random
import re
import sys
import numpy as np
import pandas as pd

DAMPING = 0.85
SAMPLES = 10000

In [110]:
def main(dir):
    
    corpus = crawl(dir)
    ranks = sample_pagerank(corpus, DAMPING, SAMPLES)
    print(f"PageRank Results from Sampling (n = {SAMPLES})")
    for page in sorted(ranks):
        print(f"  {page}: {ranks[page]:.4f}")
    ranks = iterate_pagerank(corpus, DAMPING)
    print("\n")
    print(f"PageRank Results from Iteration")
    for page in sorted(ranks):
        print(f"  {page}: {ranks[page]:.4f}")
        
def crawl(directory):
    """
    Parse a directory of HTML pages and check for links to other pages.
    Return a dictionary where each key is a page, and values are
    a list of all other pages in the corpus that are linked to by the page.
    """
    pages = dict()

    # Extract all links from HTML files
    for filename in os.listdir(directory):
        if not filename.endswith(".html"):
            continue
        with open(os.path.join(directory, filename)) as f:
            contents = f.read()
            links = re.findall(r"<a\s+(?:[^>]*?)href=\"([^\"]*)\"", contents)
            pages[filename] = set(links) - {filename}

    # Only include links to other pages in the corpus
    for filename in pages:
        pages[filename] = set(
            link for link in pages[filename]
            if link in pages
        )

    return pages

In [111]:
def transition_model(corpus, page, damping_factor):
    """
    Return a probability distribution over which page to visit next,
    given a current page.

    With probability `damping_factor`, choose a link at random
    linked to by `page`. With probability `1 - damping_factor`, choose
    a link at random chosen from all pages in the corpus.
    """
    
    names= list(corpus.keys())
    n= len(names)
    probablity={}
    if str(corpus[page]):
        
        for i in range(n):
            
            probablity[names[i]] = 1.0/n
            

    else:
        
        for i in range(n):
            
            if names[i] in corpus[page]:
                
                probablity[names[i]] = (1-damping_factor)*(1.0/n) + damping_factor* (1.0/ len(corpus[page]))
                                                                                     
            else:
                                                                                     
                probablity[names[i]] = (1-damping_factor)*(1.0/n)

    return probablity                                                                                

In [112]:
def sample_pagerank(corpus, damping_factor, n):
    """
    Return PageRank values for each page by sampling `n` pages
    according to transition model, starting with a page at random.

    Return a dictionary where keys are page names, and values are
    their estimated PageRank value (a value between 0 and 1). All
    PageRank values should sum to 1.
    """
    page_rank={}
    names= list(corpus.keys())
    num= len(names)
    
    for i in range(num):
        
        page_rank[names[i]]=0
    
    r = random.choices(names, k=1)[0]
    page_rank[r]+=1
    
    for i in range(n):
        
        transition = transition_model(corpus, r, damping_factor)
        x= list(transition.values())
        r = random.choices(names, weights=x, k=1)[0]
        page_rank[r]+=1
                
                
    
    for i in range(num):
        page_rank[names[i]]/= n
        
    return page_rank
        
         
        
        
        
        
        

In [113]:
def iterate_pagerank(corpus, damping_factor):
    """
    Return PageRank values for each page by iteratively updating
    PageRank values until convergence.

    Return a dictionary where keys are page names, and values are
    their estimated PageRank value (a value between 0 and 1). All
    PageRank values should sum to 1.
    """
    
    page_rank={}
    names= list(corpus.keys())
    num= len(names)
    
    for i in range(num):
        page_rank[names[i]]= 1.0/num
        
    flag=True
    while flag:
        
        a={} 
        for name in names:
            s= 0.0

            for x in corpus:
                if name in corpus[x]:
                    s+= page_rank[x]/ len(corpus[x])
                    
                if not corpus[x]:
                    s+= page_rank[x]/num

            a[name] = (1-damping_factor)*1.0/num + damping_factor*s
            
        flag_inner= True
        for i in range(num):
             if abs(a[names[i]]-page_rank[names[i]])>0.001:
                flag_inner= False
                break
            
        if flag_inner:
             flag=False
            
        for i in range(num):
             page_rank[names[i]]=a[names[i]]
    
    return page_rank
                
                
                

        
        
        
    

In [114]:
main("corpus0")

PageRank Results from Sampling (n = 10000)
  1.html: 0.2464
  2.html: 0.2508
  3.html: 0.2511
  4.html: 0.2518


PageRank Results from Iteration
  1.html: 0.2198
  2.html: 0.4294
  3.html: 0.2198
  4.html: 0.1311


In [115]:
main("corpus0")

PageRank Results from Sampling (n = 10000)
  bfs.html: 0.1462
  dfs.html: 0.1451
  games.html: 0.1419
  minesweeper.html: 0.1425
  minimax.html: 0.1443
  search.html: 0.1410
  tictactoe.html: 0.1391


PageRank Results from Iteration
  bfs.html: 0.1152
  dfs.html: 0.0809
  games.html: 0.2277
  minesweeper.html: 0.1180
  minimax.html: 0.1312
  search.html: 0.2090
  tictactoe.html: 0.1180


In [116]:
main("corpus1")

PageRank Results from Sampling (n = 10000)
  bfs.html: 0.1462
  dfs.html: 0.1451
  games.html: 0.1419
  minesweeper.html: 0.1425
  minimax.html: 0.1443
  search.html: 0.1410
  tictactoe.html: 0.1391


PageRank Results from Iteration
  bfs.html: 0.1152
  dfs.html: 0.0809
  games.html: 0.2277
  minesweeper.html: 0.1180
  minimax.html: 0.1312
  search.html: 0.2090
  tictactoe.html: 0.1180
